In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
import datetime
from cnn_preproc_function import *
import sklearn
from sklearn import preprocessing
from keras_metric import *
import keras
import time
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import keras
import os
from sklearn.utils import class_weight

DATA_PARAMS = dict()
DATA_PARAMS["TARGET_TO_PREDICT"] = "AUDUSD"
DATA_PARAMS["FUTURE_PERIOD_PREDICT"] = 3
DATA_PARAMS["SEQ_LEN"] = 50
DATA_PARAMS["TARGET_THRESHOLD"] = 0.001
DATA_PARAMS["FLIP"] = False

In [ ]:
#Load
df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date").sort_index()

In [ ]:
#clean and create target
df = filter_off_trading_day(df, target = DATA_PARAMS["TARGET_TO_PREDICT"], threshold = 0.1)
df = create_target(df, DATA_PARAMS["TARGET_TO_PREDICT"], DATA_PARAMS["FUTURE_PERIOD_PREDICT"], cumulative_returns)
df = classify_target(df, "target", DATA_PARAMS["TARGET_THRESHOLD"], DATA_PARAMS["FLIP"])

In [ ]:
end_split = [datetime.datetime(2011,1,1), datetime.datetime(2012,1,1), datetime.datetime(2016,1,1), datetime.datetime(2018,1,1)]

start_index, end_index = get_index_from_date(df, end_split)

target_col= "target"
x_columns = [j for j in df.columns if j != target_col]
X = df[x_columns].values
Y = df[target_col].values

#try: model.add(Lambda(lambda x: scaler.transform(x)))
scaler = sklearn.preprocessing.MinMaxScaler(feature_range = (0,1))
train_x_data = df[x_columns].iloc[start_index[0]:(end_index[0]+1)].values
scaler.fit(train_x_data)
df.loc[:,x_columns] = scaler.transform(df[x_columns])

In [ ]:
BATCH_SIZE = 256
train_data_gen = TimeseriesGenerator(X, Y,
                               length=60, sampling_rate=1,
                               batch_size=BATCH_SIZE, 
                               shuffle=True, 
                               start_index=start_index[0], end_index=end_index[0])
val_data_gen = TimeseriesGenerator(X, Y,
                               length=60, sampling_rate=1,
                               batch_size=BATCH_SIZE, 
                               shuffle=False, 
                               start_index=start_index[1], end_index=end_index[1])

In [ ]:
shape_x = data_gen[0][0].shape
print(shape_x)
print(len(data_gen))

In [ ]:
train_y = df["target"].iloc[start_index[0]:(end_index[0]+1)].values
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y),
                                                 train_y)
print(class_weights)

In [ ]:
EPOCHS = 20
PATIENCE = 20
monitor_loss = "val_loss"
mode_loss = "min"
models_folder = os.path.join("output", "models")
logs_folder = os.path.join("output", "logs")
init_dir(models_folder)
init_dir(logs_folder)

In [ ]:

t0 = time.time()

model = Sequential()
model.add(keras.layers.Reshape([shape_x[1],shape_x[2],1], input_shape=(shape_x[1],shape_x[2])))
model.add(Conv2D(128, kernel_size=(15,1), activation='relu', input_shape=(shape_x[1],shape_x[2],1)))
model.add(Conv2D(16, kernel_size=(2,1), activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

adm = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False, decay = 1e-6)
model.compile(optimizer=adm, loss='binary_crossentropy', metrics=['accuracy', precision])

tensorboard = keras.callbacks.TensorBoard(log_dir=logs_folder)
filepath = "CNN-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}-{val_precision:.4f}""
models_dir = models_folder
checkpoint = keras.callbacks.ModelCheckpoint("{}/{}.model".format(models_folder, filepath),
                                                   monitor=monitor_loss,
                                                   verbose=1,
                                                   save_best_only=True,
                                                   save_weights_only=False,
                                                   mode=mode_loss)
earlystopping = keras.callbacks.EarlyStopping(monitor=monitor_loss, 
                                              mode=mode_loss,
                                              min_delta=0, 
                                              patience=PATIENCE, 
                                              verbose=1)
history = model.fit_generator(generator=train_data_gen,
                              validation_data=val_data_gen, 
                              epochs=EPOCHS, 
                              class_weight=class_weights, 
                              callbacks=[tensorboard, checkpoint, earlystopping],             
                              verbose=1)








# history = model.fit(train_x, train_y, batch_size=256, validation_data=(val_x, val_y), epochs=EPOCHS, class_weight=class_weights, callbacks=[tensorboard, checkpoint, earlystopping])

t1 = time.time()